### Importing Basic Pacakges / ಕೆಲವೊಂದು ಅಡಿಯ ಕಟ್ಟುಗಳನ್ನು ಒಳತರಲಾಗುವುದು

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix,accuracy_score
%matplotlib inline

### Extracting the data / ತಿಳಿಹವನ್ನು ಹೊರತೆಗೆಯುವುವಿಕೆ


In [ ]:
train = pd.read_csv('../input/Kannada-MNIST/train.csv')
test = pd.read_csv('../input/Kannada-MNIST/test.csv')
dig_mnist = pd.read_csv('../input/Kannada-MNIST/Dig-MNIST.csv')

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
# Checking whether data is balanced or not.
# ಗುರುತಿನ ತಿಳಿಹುವನ್ನು ಸರಿತೂಕ ಮಾಡಿ.
values = train['label'].value_counts()
sns.barplot(values.index,values)

 **Target Classes are fairly distributed. We have 6000 samples for each numbers in kannada for training.**

In [ ]:
# Checking the rows by specifying the row number.
features = ["pixel{}".format(pixel_num) for pixel_num in range (0,784)]# Stores the pixel array from pixel1 to pixel785
rows_to_examine = 9
image_data = np.reshape(train[features][rows_to_examine:rows_to_examine + 1].to_numpy(),(28,28))
plt.imshow(image_data,cmap='gray')
plt.show()

## Data Preparation / ತಿಳಿಹುವಿನ ಸಮೆಯುವಿಕೆ

#### Spliting train datasets for x and y variables / ಪಳಗಿಸುವ ತಿಳಿಹುಗಳಿಂದ ಗುರುತು(y) ಮತ್ತು ತಾನ್ಮೆ(x) ಮಾರ್ಪುಕಗಳಿಗೆ ಬೇರೆ ಮಾಡುವುದು


In [ ]:
x = train.drop('label',axis = 1)
y = train['label']
print(x.shape)
print(y.shape)

#### Splitting Datasets into Train and Test / ತಿಳಿಹುವನ್ನು ಪಳಗಿಸುವ ಮತ್ತು ಒರೆತಗಳಿಗೆ ಬೇರ್ಪಡಿಸುವಿಕೆ.

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=46)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
# Converting DataFrame to array
x_train = x_train.to_numpy()
x_test = x_test.to_numpy()

#### Normalization / ಎಂದಿನಂತಾಗಿಸುವುದು¶


In [ ]:
# Reshape and Normalization
x_train = x_train.reshape(x_train.shape[0],28,28,1).astype('float64')/255
x_test = x_test.reshape(x_test.shape[0],28,28,1).astype('float64')/255

In [ ]:
print(x_train.shape)
print(x_test.shape)


## Visualization / ನೋಟ


In [ ]:

class_names = [0,1,2,3,4,5,6,7,8,9]
plt.figure(figsize = (15,15))
for k in range(10):
    xx = x_train[y_train == k]
    yy = y_train[y_train == k].reset_index()['label']
    for i in range(10):
        plt.subplot(10,10,k * 10 + i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(xx[i][:,:,0],cmap = 'gray')
        label_index = int(yy[i])
        plt.title('{}. {}'.format(k,class_names[label_index]))
plt.show()        



#### Encoding Label Values / ಗುರುತು ಬೆಲೆಯನ್ನು ಗುಟ್ಟುಬರೆಯುವಿಕೆ

In [ ]:
# to_categorical is used to perform one-hot encoding for label values
from keras.utils import np_utils
y_train1 = np_utils.to_categorical(y_train,10)
y_test1 = np_utils.to_categorical(y_test,10)

In [ ]:
 plt.imshow(x_train[5])

#### Note - Above image shows 5/೫ in Kannada for a row 5 selection.
### ಮೇಲಗಡೆಯ ತಿಟ್ಟದಲ್ಲಿ ಕಾಣುವ ಎಣಿಕೆ ಕನ್ನಡ ನುಡಿಯಲ್ಲಿ ೫ ಆಗಿದೆ.

#### Data Augumentation / ತಿಳಿಹುವನ್ನು ಹೆಚ್ಚಿಸುವಿಕೆ

In [ ]:
# create more sample images using ImageDataGenerator.This ensures we have more data to train on. More info about function here: https://keras.io/preprocessing/image/
from keras.preprocessing.image import ImageDataGenerator
imagegen = ImageDataGenerator(
        featurewise_center=False, 
        samplewise_center=False, 
        featurewise_std_normalization=False, 
        samplewise_std_normalization=False,
        zca_whitening=False, 
        rotation_range=9, 
        zoom_range = 0.25,
        width_shift_range=0.25,
        height_shift_range=0.25, 
        horizontal_flip=False, 
        vertical_flip=False)

imagegen.fit(x_train)

In [ ]:
# importing DL packages / ಆಳವಾದ ಕಲಿಕೆಯಲ್ಲಿ ಬಳಸುವ ಕೆಲವೊಂದು ಕಟ್ಟುಗಳನ್ನು ಒಳತರುವುದು
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Conv2D,MaxPooling2D,Dense,Dropout


## Building CNN Model (With dropout) // ಸಿ ಎನ್ ಎನ್ ಎತ್ತುಗೆ ಕಟ್ಟುವಿಕೆ 


In [ ]:
tf.random.set_seed(100)
tf.keras.backend.clear_session()
model = None
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),input_shape=(28,28,1),activation = 'relu'))
model.add(MaxPooling2D(pool_size = 2))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size = 2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128,activation='relu'))# First Hidden Layer with fully connected
model.add(Dense(64,activation='relu'))# First Hidden Layer with fully connected
model.add(Dense(10,activation='softmax')) # Output layer for 10 classification

### Train the model / ಎತ್ತುಗೆಯನ್ನು ಪಳಗಿಸು¶


In [ ]:
model.compile(loss='categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
model.summary()

In [ ]:
history = model.fit_generator(imagegen.flow(x_train,y_train1,batch_size=128),epochs=30,validation_data=(x_test,y_test1),verbose = 1)

### Evaluation and Prediction / ಬೆಲೆತೂಗುವಿಕೆ ಮತ್ತು ಮುನ್ಹೊಳಹು

In [ ]:
test_score = model.evaluate(x_test,y_test1)
train_score = model.evaluate(x_train,y_train1)
print('\n Train Accuracy : %4f' % (train_score[1]))
print('\n Test Accuracy : %4f' % (test_score[1]))


### Accuray and Error Graph / ಸರಿಮೆ ಮತ್ತು ತಪ್ಪುಗಳ ಮಾರ್ಪುತಿಟ್ಟ¶


In [ ]:
train_accuracy = history.history['accuracy']
train_loss = history.history['loss']

val_accuracy = history.history['val_accuracy']
val_loss = history.history['val_loss']

epochs = range(len(train_accuracy))
plt.plot(epochs,train_accuracy,'o-g',label='Train')
plt.plot(epochs,val_accuracy,'o-b',label='Validation/Test')
plt.title('Accuracy')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
#plt.figure()
plt.show()

plt.plot(epochs,train_loss,'o-g',label='Train')
plt.plot(epochs,val_loss,'o-r',label = 'Validation/Test')
plt.title('Loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

### Confusion Matrix / ಗೊಂದಲ,ಎಕ್ಕ-ಸಕ್ಕ ಅಣೆ

In [ ]:
y_test_pred = model.predict(x_test)
y_test_pred = np.argmax(y_test_pred,axis = 1)
y_test_pred

In [ ]:
y_test1= np.argmax(y_test1,axis = 1)
y_test1

In [ ]:
conf = confusion_matrix(y_test1,y_test_pred)
conf = pd.DataFrame(conf,index = range(0,10),columns = range(0,10))
conf

In [ ]:
plt.figure(figsize = (7,7))
sns.heatmap(conf,annot=True,cmap = plt.cm.Greens )
plt.show()

#### Further investigation / ಮುಂದುವರಿದ ಆರಯ್ಯಿಕೆ

In [ ]:
# Check Incorrectly Misclassified / ತಪ್ಪಾಗಿ ಹಂಚಿಕೆಯಾಗಿರುವುದನ್ನು ನೋಡಿ
x = (y_test_pred - y_test1 != 0).tolist()
x = [ i for i,l in enumerate(x) if l != False]
fig,ax = plt.subplots(1,4,sharey=False,figsize = (12,12))
for i in range(4):
    ax[i].imshow(x_test[x[i]][:,:,0])
    ax[i].set_xlabel('Actual {}, Predicted {}'
                     .format(y_test1[x[i]],y_test_pred[x[i]]))
plt.show()


In [ ]:
# Check Correctly Classified / ಸರಿಯಾಗಿ ಹಂಚಿಕೆಯಾಗಿರುವುದನ್ನು ನೋಡಿ
x = (y_test_pred - y_test1 == 0).tolist()
x = [ i for i,l in enumerate(x) if l != False]
fig,ax = plt.subplots(1,4,sharey=False,figsize = (12,12))
for i in range(4):
    ax[i].imshow(x_test[x[i]][:,:,0])
    ax[i].set_xlabel('Actual {}, Predicted {}'
                     .format(y_test1[x[i]],y_test_pred[x[i]]))
plt.show()

### Submission / ಸಲ್ಲಿಕೆ

In [ ]:
test.head()

In [ ]:
test_id = test['id']
test = test.drop(['id'],axis=1)
test = test.to_numpy()
test = test.reshape(test.shape[0],28,28,1).astype('float64')/255
test.shape

In [ ]:
y_pred = model.predict(test)
y_pred = np.argmax(y_pred,axis = 1) # Changing the prediction into labels

In [ ]:
submission = pd.DataFrame({ 'id': test_id,
                            'label': y_pred })
submission.to_csv("submission.csv", index=False)
submission.head()


### CNN model is better for image classification.
### CNN test accuracy : 99%
### ತಿಟ್ಟ ಹಂಚಿಕೆಗೆ CNN ಎತ್ತುಗೆ ಒಳ್ಳೆಯದಾಗಿದೆ.
### CNN ಒರೆತದ ಸರಿಮೆ: ೯೯%
    
### Please upvote this notebook if you find anything helpful !! Thank you
### ಇ ನನ್ನ ಹೊತ್ತಿಗೆಯಿಂದ ನಿಮಗೇ ಏನಾದರು ನೆರವಾಗಿದ್ದಾರೆ ದಯವಿಟ್ಟು ಮೇಲೆ ಬಲಗಡೆಯಲ್ಲಿ ಕಾಣುವ upvote ಬಟನ್ ಅನ್ನು ಹೊತ್ತಿ. ನನ್ನಿ !!